In [ ]:
import tensorflow as tf
# tf.enable_eager_execution()

In [ ]:
from networks import SparseAutoEncoder
sae = SparseAutoEncoder(resolution=128, ae_dims=32, d_ch_dims=10, e_ch_dims=21)

In [ ]:
optimizer = tf.keras.optimizers.Adam()
mse_loss_fn = tf.keras.losses.MeanSquaredError()
loss_metric = tf.keras.metrics.Mean()

In [ ]:
(x_train, _), _ = tf.keras.datasets.cifar10.load_data()
x_train = x_train.reshape(50000, 32, 32, 3).astype('float32') / 255
x_train = tf.image.resize_images(x_train, (128, 128))

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(4)

In [ ]:
# Iterate over epochs.
for epoch in range(3):
    print('Start of epoch %d' % (epoch,))
    
    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = sae(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
        
        grads = tape.gradient(loss, sae.trainable_weights)
        
        loss_metric(loss)
        
        if step % 100 == 0:
            print('step %s: mean loss = %s' % (step, loss_metric.result()))
            